# 02467 Assignment 2
s204085 Cornelius Erichs & s204076 Natasha Hougaard

#### GitHub repository
Here is our repository on GitHub: https://github.com/natasha0301/Assigment2-CSS

The commit history does not represent the true story of collaboration, as it was worked on by both of us at the sametime while sharing our screen on zoom, or looking over the other persons shoulder.

## Part 1: Mixing Patterns and Assortativity

For each node, compute the fraction of edges that connect to a node that works in the same top field. Find the average value across all nodes.

In [ ]:
import json
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)
G = read_json_file("data_total.json")
def same_field(G):
    same_field_fractions = []
    for node in G.nodes():
        same_field_neighbors = 0
        total_neighbors = 0
        
        for neighbor in G.neighbors(node):
            if G.nodes[neighbor]["field"] == G.nodes[node]["field"]:
                same_field_neighbors += 1
            total_neighbors += 1
        
        if total_neighbors > 0:
            same_field_fraction = same_field_neighbors / total_neighbors
        else:
            same_field_fraction = 0
        same_field_fractions.append(same_field_fraction)
    return same_field_fractions

same_field_fractions = same_field(G)
print(np.mean(same_field(G)))

## Part 2: Communities

**EXERCISE 2: ZACHARYS'S KARATE CLUB**

In this exercise, we will work on Zarachy's karate club graph (refer to the Introduction of Chapter 9). The dataset is available in NetworkX, by calling the function karate_club_graph

1. Visualize the graph using netwulf. Set the color of each node based on the club split

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from netwulf import visualize

G = nx.karate_club_graph()

#First we set the color of each node based on the club split
colors = ['red' if G.nodes[n]['club'] == 'Mr. Hi' else 'blue' for n in G.nodes()]

#Then we add the colors to the node attributes
for i, n in enumerate(G.nodes()):
    G.nodes[n]['color'] = colors[i]

#Visualize the graph
visualize(G)

2. Write a function to compute the modularity of a graph partitioning using equation 9.12 in the book: 

$M=\sum_{c=1}^{n_c}\left[\frac{L_c}{L}-(\frac{k_c}{2L})^2\right]$ 



The function should take a networkX Graph and a partitioning as inputs and return the modularity.

In [ ]:
"""
def compute_modularity(G,partitioning):
    L = G.number_of_edges()
    M = 0
    communities = set(partitioning.values())

    for community in communities:
        nodes_in_community = [node for node, community_id in partitioning.items() if community_id == community]
        subgraph = G.subgraph(nodes_in_community)
        k_c = sum(dict(subgraph.degree()).values())
        L_c = subgraph.number_of_edges()
        M += L_c / L - (k_c / (2 * L)) ** 2

    return M
"""

3. Explain in your own words the concept of modularity **REFRASE**

Modularity is a measure of the structure of a network, which is a collection of nodes (also known as vertices) and the edges (also known as links) that connect them. The concept of modularity is based on the idea that a well-connected network can be divided into groups of nodes, known as communities or clusters, where the nodes within each group are more densely connected to each other than to nodes outside the group.

Modularity is a measure of how well a network can be divided into such communities. Specifically, modularity quantifies the difference between the number of edges within communities and the expected number of edges within communities if the edges were placed randomly, while preserving the degree distribution of the nodes. In other words, modularity measures the extent to which the edges in a network are concentrated within communities, rather than being distributed randomly throughout the network.

Modularity is typically expressed as a number between -1 and 1. A high positive value of modularity indicates that the network is highly modular, with many densely connected communities, while a low or negative value indicates that the network is less modular, with communities that are not well-defined. The concept of modularity has many applications in network science, including community detection, network visualization, and network comparison.

4. Compute the modularity of the Karate club split partitioning using the function you just wrote

In [ ]:
import networkx as nx

def compute_modularity(G,partitioning):
    L = G.number_of_edges()
    M = 0
    communities = set(partitioning.values())

    for community in communities:
        nodes_in_community = [node for node, community_id in partitioning.items() if community_id == community]
        subgraph = G.subgraph(nodes_in_community)
        k_c = sum(dict(subgraph.degree()).values())
        L_c = subgraph.number_of_edges()
        M += L_c / L - (k_c / (2 * L)) ** 2

    return M

#We start by loading the Karate club graph
G = nx.karate_club_graph()

#First we get the club split partitioning from the node attributes
club_split = nx.get_node_attributes(G, "club")

#Then we compute the modularity of the partitioning by using the function above
modularity = compute_modularity(G, club_split)

print(f"The modularity of the Karate club split partitioning using the function is {modularity:.3f}")

5. We will now perform a small randomization experiment to assess if the modularity you just computed is statitically different from 0. To do so, we will implement the double edge swap algorithm. Given a network G, this algorithm creates a new network, such that each node has exactly the same degree as in the original network, but different connections. Here is how the algorithm works.

    a. Create an identical copy of your original network.
    
    b. Consider two edges in your new network (u,v) and (x,y), such that u!=v and v!=x.
    
    c. If none of edges (u,y) and (x,v) exists already, add them to the network and remove edges (u,v) and (x,y).
    
Repeat steps b. and c. to achieve at least N swaps (I suggest N to be larger than the number of edges).

## Part 3: TF-IDF